In [1]:
import json
import pandas as pd

In [2]:
# Step 1: Read the content of the first JSON file
with open('NER_TRAIN/NER_TRAIN_PREAMBLE.json', 'r') as file1:
    data = json.load(file1)

In [3]:
# Step 1: Read the content of the first JSON file
with open('NER_DEV/NER_DEV_PREAMBLE.json', 'r') as file1:
    data2 = json.load(file1)

In [4]:
len(data)

1560

In [5]:
training_data = {'classes' : ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER"], 'annotations' : []}
for example in data:
    temp_dict = {}
#     print(example['data']['text'])
#     ppp
    text = example['data']['text']
    temp_dict['text'] = text
#     print(temp_dict['text'])
    temp_dict['entities'] = []
#     print(example['annotations'][0]['result'])
#     ppp
    for annotation in example['annotations'][0]['result']:
        start = annotation['value']['start']
        end = annotation['value']['end']
        label = annotation['value']['labels'][0].upper()
        temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
    
print(training_data['annotations'][156])

{'text': 'Petitioner:\nAddagada Raghavamma And Anr.\n\n\tVs.\n\nRespondent:\nAddagada Chenchamma And Anr.\n\nDate Of Judgment', 'entities': [(12, 31, 'PETITIONER'), (60, 79, 'RESPONDENT')]}


In [6]:
test_data = {'classes' : ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER"], 'annotations' : []}
for example in data2:
    temp_dict = {}
#     print(example['data']['text'])
#     ppp
    text = example['data']['text']
    temp_dict['text'] = text
#     print(temp_dict['text'])
    temp_dict['entities'] = []
#     print(example['annotations'][0]['result'])
#     ppp
    for annotation in example['annotations'][0]['result']:
        start = annotation['value']['start']
        end = annotation['value']['end']
        label = annotation['value']['labels'][0].upper()
        temp_dict['entities'].append((start, end, label))
    test_data['annotations'].append(temp_dict)
    
print(test_data['annotations'][15])

{'text': "High Court Of Judicature At Allahabad\n \n \n\n                                                                             A.F.R.\n \n                                                 \t          Reserved on 07.10.2021\n \n\t\t\t\t\t\t          Delivered on 13.12.2021\n \n\n \n\n \nCase :- Writ - C No. - 59863 of 2015\n \nPetitioner :- Sun Tower Residents Welfare Association\n \nRespondent :- Ghaziabad Development Authority through its Vice Chairman and 2 Others\n \nCounsel for Petitioner :- Prashant,Abhijeet Mukherji,Prashant,S.K. Pal\n \nCounsel for Respondent :- Ram Bilas Yadav,Anoop Tivedi,Anoop Trivedi (Senior Adv.),Himanshu Tyagi,Kartikeya Saran,Rahul Agarwal,Rakesh Kumar Singh,S.Shekhar,Vrindavan Mishra\n \n\n \n                       Connected with \n \n\n \nCase :- Writ - C No. - 11072 of 2017\n \nPetitioner :- Sun Tower Residents Welfare Association through Vice President\n \nRespondent :- Ghaziabad Development Authority through its Vice Chairman and 2 Others\n \nCo

In [7]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

# nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm")
doc_bin = DocBin()

In [9]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("./train.spacy") # save the docbin object

100%|██████████████████████████████████████| 1560/1560 [00:03<00:00, 506.05it/s]


In [10]:
from spacy.util import filter_spans

for test_example in tqdm(test_data['annotations']): 
    text = test_example['text']
    labels = test_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("./dev.spacy") # save the docbin object

100%|████████████████████████████████████████| 125/125 [00:00<00:00, 453.10it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [11]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python3 -m spacy download en_core_web_lg

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.4.1/en_core_web_lg-3.4.1-py3-none-any.whl#egg=en_core_web_lg==3.4.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.6 MB/s eta 0:00:0000:0100:02
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-c

In [11]:
!python3 -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2023-06-12 17:13:46,560] [INFO] Set up nlp object from config
[2023-06-12 17:13:46,565] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-12 17:13:46,568] [INFO] Created vocabulary
[2023-06-12 17:13:47,637] [INFO] Added vectors: en_core_web_lg
[2023-06-12 17:13:49,806] [INFO] Finished initializing nlp object
[2023-06-12 17:14:09,196] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.41    0.04    0.05    0.03    0.00
  0     200        832.15   2518.92    1.93    3.25    1.37    0.02

In [15]:
!python3 -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy


ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2023-06-12 21:27:10,238] [INFO] Set up nlp object from config
[2023-06-12 21:27:10,244] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-12 21:27:10,244] [INFO] Resuming training for: ['ner']
[2023-06-12 21:27:10,247] [INFO] Created vocabulary
[2023-06-12 21:27:11,580] [INFO] Added vectors: en_core_web_lg
[2023-06-12 21:27:12,484] [INFO] Finished initializing nlp object
[2023-06-12 21:27:20,571] [INFO] Initialized pipeline components: ['tok2vec']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.78      6.09    0.00    0.00    0.00    0.00
  0     20